In [ ]:
# !pip install gensim

In [1]:
import pandas as pd
from scipy import spatial
from gensim.models import KeyedVectors
import numpy as np
from numpy import dot
from numpy.linalg import norm

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

/Users/wizard/anaconda3/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
train = pd.read_csv('task3_training.tsv',sep='\t')
valid = pd.read_csv('task3_validation.tsv',sep='\t')
train.dropna(axis=1, how='all',inplace=True)
valid.dropna(axis=1, how='all',inplace=True)

# meddra_codes = pd.read_excel('patient-friendly_term_list_v25.0.xlsx')
tested_valid = pd.read_csv('predicted_with_data.csv',sep='|')

In [3]:
temp = train.append(valid,ignore_index=True)[['meddra_code','meddra_term']]

temp.drop_duplicates(inplace=True)

temp.reset_index(inplace=True,drop=True)
temp.drop([0],inplace=True)
temp.reset_index(inplace=True,drop=True)
meddra_codes = temp.copy()

In [ ]:
# Checking for whether meddra codes are there in test and eval
# valid_meddra_codes = valid['meddra_code'].unique()

# for meddra_code in train['meddra_code'].unique():
#     if meddra_code in valid_meddra_codes:
#         print(meddra_code)

In [ ]:
# !wget --help

In [4]:
# EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
# !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [5]:
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

- Sample Cosine-Similarity

In [8]:
#@title Sample Cosine Similarity
word1 = 'died'
word2 = 'stevens johnson syndrome'

# model.wmdistance(row[-1],row[5])
list1 = [model[word] for word in [word1]]
list2 = [model[word] for word in word2.split()]

list1_avg = np.average(list1,axis=0)
list2_avg = np.average(list2,axis=0)

## Method 1 to calculate similarity
result = 1 - spatial.distance.cosine(list1_avg, list2_avg)

## Method 2 to calculate similarity
cos_sim = dot(list1_avg, list2_avg)/(norm(list1_avg)*norm(list2_avg))

In [9]:
cos_sim, result

(0.09778746, 0.09778745472431183)

- Creating the meddra_codes_index along with the word2vec averages

In [10]:

meddra_codes_llt_dict = {}
meddra_codes_reverse = {}
# for code, LLT_code in zip(meddra_codes['LLT'],meddra_codes['LLT code']):
for code, LLT_code in zip(meddra_codes['meddra_term'],meddra_codes['meddra_code']):
    try:
        # meddra_codes_reverse[LLT_code] = code
        meddra_codes_reverse[LLT_code] = code
        list1 = [model[word] for word in code.split()]
        list1_avg = np.average(list1,axis=0)
        meddra_codes_llt_dict[LLT_code] = list1_avg
    except:
        pass

In [11]:
tested_valid.head()

,Unnamed: 0,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,prediction_extractions
0,0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,[]
1,1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs",10024668.0,liver damage,[]
2,2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003731.0,attention deficit disorder,[]
3,3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003738.0,attention impaired,[]
4,4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills,10011906.0,death,['died']


In [12]:
def meddra_map(x):
    if x==0:
        return 0
    else:
        try:
            for item in x:
                list1 = [model[word.lower()] for word in item.split()]
                list1_avg = np.average(list1,axis=0)
                current_best = 0
                current_best_catergorization = 0
                for key, item in meddra_codes_llt_dict.items():
                    # print(key)
                    list2_avg = item
                    # cos_sim = dot(list1_avg, list2_avg)/((norm(list1_avg)*norm(list2_avg)))
                    result = 1 - spatial.distance.cosine(list1_avg, list2_avg)
                    if result > current_best:
                        # print(cos_sim)
                        current_best = result
                        current_best_catergorization = meddra_codes_reverse[key]
            return current_best_catergorization
        except:
            return 0
            

In [13]:
# final_result.head()

In [14]:
final_result = tested_valid.copy(deep=True)

final_result['extraction'] = final_result['extraction'].fillna(0)
final_result['meddra_term'] = final_result['meddra_term'].fillna(0)

final_result['true_extraction_label'] = final_result['extraction'].apply(lambda x: 1 if x!=0 else 0)
final_result['true_meddra_label'] = final_result['meddra_term'].apply(lambda x: 1 if x!=0 else 0)

final_result['pred_meddra_term'] = final_result.prediction_extractions.apply(meddra_map)

strict_pred_extraction_label = []
relaxed_pred_extraction_label = []
pred_meddra_label = []

count = 1
for index, row in final_result.iterrows():
    # print(row)
    if row['true_extraction_label'] == 1 and len(row['prediction_extractions']) == 0:
        strict_pred_extraction_label.append(0)
        relaxed_pred_extraction_label.append(0)
    else:
        # Strict
        strict_flag = False
        if row['extraction'] == 0 and row['prediction_extractions'] != []:
            strict_pred_extraction_label.append(1)
        else:
            for item in row['prediction_extractions']:
                if row['extraction'] == item:
                    strict_pred_extraction_label.append(1)
                    strict_flag = True
                    break
            if not strict_flag:
                strict_pred_extraction_label.append(0)

        # Relaxed
        relaxed_flag = False
        if row['extraction'] == 0 and row['prediction_extractions'] != []:
            relaxed_pred_extraction_label.append(1)
        else:
            if row['extraction'] in row['prediction_extractions']:
                relaxed_pred_extraction_label.append(1)
                relaxed_flag = True
            if not relaxed_flag:
                relaxed_pred_extraction_label.append(0)

# print(count)
final_result['strict_pred_extraction_label'] = strict_pred_extraction_label
final_result['relaxed_pred_extraction_label'] = relaxed_pred_extraction_label

In [15]:
for index, row in final_result.iterrows():
    # MedDRA
    meddra_flag = False
    # print(row)
    # if row['pred_meddra_term'] == 0:
    #     pred_meddra_label.append(0)
    if row['meddra_term'] == 0 and row['pred_meddra_term'] == 0:
        pred_meddra_label.append(1)
    elif row['meddra_term'] == 0 and row['pred_meddra_term'] != 0:
        pred_meddra_label.append(1)
    elif row['meddra_term'] != 0 and row['pred_meddra_term'] == 0:
        pred_meddra_label.append(0)
    else:
        if row['meddra_term'] in row['pred_meddra_term']:
            pred_meddra_label.append(1)
            meddra_flag = True
        else:
            pred_meddra_label.append(0)

In [16]:
final_result['pred_meddra_label'] = pred_meddra_label

In [17]:
final_result

,Unnamed: 0,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,prediction_extractions,true_extraction_label,true_meddra_label,pred_meddra_term,strict_pred_extraction_label,relaxed_pred_extraction_label,pred_meddra_label
0,0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,[],1,1,0,0,0,0
1,1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs",10024668.0,liver damage,[],1,1,0,0,0,0
2,2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003731.0,attention deficit disorder,[],1,1,0,0,0,0
3,3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003738.0,attention impaired,[],1,1,0,0,0,0
4,4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills,10011906.0,death,['died'],1,1,0,0,1,0
5,5,332542581760204800,41.0,45.0,ADR,died,cipro,"@gastromom the only pt of mine who ever died was one age 21 profound autism chronic underwt &amp; gi issues, given lots of cipro. terrible",10011906.0,death,[],1,1,0,0,0,0
6,6,343944308828811264,66.0,79.0,ADR,tendon damage,cipro,owww. i hurt my foot. &amp; am concerned. 1st warning on cipro is tendon damage. :-/ it certainly wasn't a rupture. i'm sure i'm just paranoid.,10043242.0,tendon injury,[],1,1,0,0,0,0
7,7,346700255829561344,NaN,NaN,NaN,0,cipro,@deadfanclub @danadelany i have a penicillin/ keflex allergy so my dr writes me 10 days of zpack or i'd still be sick too! sometimes cipro,NaN,0,['sick'],0,0,0,1,1,1
8,8,348120419921952770,NaN,NaN,NaN,0,cipro,@joshuagates you forgot the immodium for when the cipro kills all the good bacteria. :) #travelskills,NaN,0,[],0,0,0,1,1,1
9,9,348644040444637184,26.0,32.0,ADR,dreams,Robitussin,@bilgeebiri oh robitussin dreams are notorious. although what will really turn 'em into a bunuel movie is cipro.,10004969.0,bizarre dreams,[],1,1,0,0,0,0


In [ ]:
# final_result['pred_meddra_label'] = pred_meddra_label

In [18]:
final_result.shape

(560, 17)

In [ ]:
# final_result.isnull().sum()

In [20]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

In [69]:
# # # Relaxed - NER
# print(classification_report(final_result['true_extraction_label'], final_result['relaxed_pred_extraction_label']))
# print("F1 score: ", f1_score(final_result['true_extraction_label'], final_result['relaxed_pred_extraction_label']))
# print("Precision score: ", precision_score(final_result['true_extraction_label'], final_result['relaxed_pred_extraction_label']))
# print("Recall score: ", recall_score(final_result['true_extraction_label'], final_result['relaxed_pred_extraction_label']))
# print("Confusion Matrix:\n", confusion_matrix(final_result['true_extraction_label'], final_result['relaxed_pred_extraction_label']))

In [70]:
# # # Strict - NER
# print(classification_report(final_result['true_extraction_label'], final_result['strict_pred_extraction_label']))
# print("F1 score: ", f1_score(final_result['true_extraction_label'], final_result['strict_pred_extraction_label']))
# print("Precision score: ", precision_score(final_result['true_extraction_label'], final_result['strict_pred_extraction_label']))
# print("Recall score: ", recall_score(final_result['true_extraction_label'], final_result['strict_pred_extraction_label']))

In [71]:
# # # MedDRA
# print(classification_report(final_result['true_meddra_label'], final_result['pred_meddra_label']))
# print("F1 score: ", f1_score(final_result['true_meddra_label'], final_result['pred_meddra_label']))
# print("Precision score: ", precision_score(final_result['true_meddra_label'], final_result['pred_meddra_label']))
# print("Recall score: ", recall_score(final_result['true_meddra_label'], final_result['pred_meddra_label']))

In [24]:
tested_valid

,Unnamed: 0,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,prediction_extractions
0,0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,[]
1,1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs",10024668.0,liver damage,[]
2,2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003731.0,attention deficit disorder,[]
3,3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003738.0,attention impaired,[]
4,4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills,10011906.0,death,['died']
5,5,332542581760204800,41.0,45.0,ADR,died,cipro,"@gastromom the only pt of mine who ever died was one age 21 profound autism chronic underwt &amp; gi issues, given lots of cipro. terrible",10011906.0,death,[]
6,6,343944308828811264,66.0,79.0,ADR,tendon damage,cipro,owww. i hurt my foot. &amp; am concerned. 1st warning on cipro is tendon damage. :-/ it certainly wasn't a rupture. i'm sure i'm just paranoid.,10043242.0,tendon injury,[]
7,7,346700255829561344,NaN,NaN,NaN,NaN,cipro,@deadfanclub @danadelany i have a penicillin/ keflex allergy so my dr writes me 10 days of zpack or i'd still be sick too! sometimes cipro,NaN,NaN,['sick']
8,8,348120419921952770,NaN,NaN,NaN,NaN,cipro,@joshuagates you forgot the immodium for when the cipro kills all the good bacteria. :) #travelskills,NaN,NaN,[]
9,9,348644040444637184,26.0,32.0,ADR,dreams,Robitussin,@bilgeebiri oh robitussin dreams are notorious. although what will really turn 'em into a bunuel movie is cipro.,10004969.0,bizarre dreams,[]


- Part1 - NER 

In [25]:
import ast
tested_valid['prediction_extractions'] = tested_valid['prediction_extractions'].apply(lambda x: ast.literal_eval(x))


# Unrolling
new_test_valid = pd.DataFrame()

for idx,ser in tested_valid.iterrows():
    # print(ser)
    if len(ser['prediction_extractions'])>0:
        for prediction in ser['prediction_extractions']:
            ser['main extraction prediction'] = prediction
            new_test_valid = new_test_valid.append(ser,ignore_index=True)
    else:
        ser['main extraction prediction'] = np.nan
        new_test_valid = new_test_valid.append(ser,ignore_index=True)


new_test_valid['main extraction prediction'] = new_test_valid['main extraction prediction'].fillna(0)
new_test_valid['extraction'] = new_test_valid['extraction'].fillna(0)

new_test_valid['main extraction prediction'] = new_test_valid['main extraction prediction'].astype(str)
new_test_valid['extraction'] = new_test_valid['extraction'].astype(str)

new_test_valid['meddra_term'] = new_test_valid['meddra_term'].astype(str)
new_test_valid['meddra_term'] = new_test_valid['meddra_term'].astype(str)

In [48]:
new_test_valid.head(50)

,Unnamed: 0,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,prediction_extractions,main extraction prediction,meddra classification prediction,soft_medra_classification,hard_medra_classfication,extraction binarized,meddra_term binarized,soft_ext_classification,hard_ext_classfication,true_ext
0,0.0,3.323175e+17,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,[],0,0,0,0,1,1,0,0,1
1,1.0,3.478062e+17,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs",10024668.0,liver damage,[],0,0,0,0,1,1,0,0,1
2,2.0,3.503361e+17,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003731.0,attention deficit disorder,[],0,0,0,0,1,1,0,0,1
3,3.0,3.503361e+17,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003738.0,attention impaired,[],0,0,0,0,1,1,0,0,1
4,4.0,3.325407e+17,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills,10011906.0,death,[died],died,death,1,1,1,1,1,1,1
5,5.0,3.325426e+17,41.0,45.0,ADR,died,cipro,"@gastromom the only pt of mine who ever died was one age 21 profound autism chronic underwt &amp; gi issues, given lots of cipro. terrible",10011906.0,death,[],0,0,0,0,1,1,0,0,1
6,6.0,3.439443e+17,66.0,79.0,ADR,tendon damage,cipro,owww. i hurt my foot. &amp; am concerned. 1st warning on cipro is tendon damage. :-/ it certainly wasn't a rupture. i'm sure i'm just paranoid.,10043242.0,tendon injury,[],0,0,0,0,1,1,0,0,1
7,7.0,3.467003e+17,NaN,NaN,NaN,0,cipro,@deadfanclub @danadelany i have a penicillin/ keflex allergy so my dr writes me 10 days of zpack or i'd still be sick too! sometimes cipro,NaN,nan,[sick],sick,feeling sick,0,0,0,0,0,0,1
8,8.0,3.481204e+17,NaN,NaN,NaN,0,cipro,@joshuagates you forgot the immodium for when the cipro kills all the good bacteria. :) #travelskills,NaN,nan,[],0,0,1,1,0,0,1,1,1
9,9.0,3.486440e+17,26.0,32.0,ADR,dreams,Robitussin,@bilgeebiri oh robitussin dreams are notorious. although what will really turn 'em into a bunuel movie is cipro.,10004969.0,bizarre dreams,[],0,0,0,0,1,1,0,0,1


In [52]:
new_test_valid['true_ext'] = [1 if token != 0 else 0 for token in new_test_valid.extraction]

In [59]:
def new_temp(x):
    # print(type(x))
    if x=='0':
        return 0
    else:
        try:
            list1 = [model[word.lower()] for word in x.split()]
            list1_avg = np.average(list1,axis=0)
            current_best = 0
            current_best_catergorization = 0
            for key, item in meddra_codes_llt_dict.items():
                # print(key)
                list2_avg = item
                # cos_sim = dot(list1_avg, list2_avg)/((norm(list1_avg)*norm(list2_avg)))
                result = 1 - spatial.distance.cosine(list1_avg, list2_avg)
                if result > current_best:
                    # print(cos_sim)
                    current_best = result
                    current_best_catergorization = meddra_codes_reverse[key]
            return current_best_catergorization
        except:
            return 0

# new_test_valid['meddra classification prediction'] = new_test_valid['main extraction prediction'].apply(new_temp)

soft_medra_classification = []
hard_medra_classfication = []
for actual, predicted in zip(new_test_valid['meddra_term'],new_test_valid['meddra classification prediction']):
    breaker = True
    if predicted == 0:
        predicted = 'nan'
    for actual_term in str(actual).split():
        if actual_term in str(predicted).split():
            soft_medra_classification.append(1)
            breaker = False
            break
    if breaker:
        soft_medra_classification.append(0)

    if actual == predicted:
        hard_medra_classfication.append(1)
    else:
        hard_medra_classfication.append(0)

new_test_valid['soft_medra_classification'] = soft_medra_classification
new_test_valid['hard_medra_classfication'] = hard_medra_classfication

soft_ext_classification = []
hard_ext_classfication = []
for actual, predicted in zip(new_test_valid[new_test_valid.true_ext == 1]['extraction'],new_test_valid[new_test_valid.true_ext == 1]['main extraction prediction']):
    breaker = True
    if predicted == 0:
        predicted = 'nan'
    for actual_term in str(actual).split():
        if actual_term in str(predicted).split():
            soft_ext_classification.append(1)
            breaker = False
            break
    if breaker:
        soft_ext_classification.append(0)

    if actual == predicted:
        hard_ext_classfication.append(1)
    else:
        hard_ext_classfication.append(0)

new_test_valid['soft_ext_classification'] = soft_ext_classification
new_test_valid['hard_ext_classfication'] = hard_ext_classfication


# soft_medra_classification = []
# hard_medra_classfication = []
# for actual, predicted in zip(new_test_valid['extraction'],new_test_valid['main extraction prediction']):
#     breaker = True
#     if predicted == 0:
#         predicted = 'nan'
#     for actual_term in str(actual).split():
#         if actual_term in str(predicted).split():
#             soft_medra_classification.append(1)
#             breaker = False
#             break
#     if breaker:
#         soft_medra_classification.append(0)

#     if actual == predicted:
#         hard_medra_classfication.append(1)
#     else:
#         hard_medra_classfication.append(0)

# new_test_valid['soft_medra_extraction'] = soft_medra_classification
# new_test_valid['hard_medra_extraction'] = hard_medra_classfication

new_test_valid['extraction binarized'] = new_test_valid['extraction'].apply(lambda x: 1 if x!='0' else 0)
new_test_valid['meddra_term binarized'] = new_test_valid['meddra_term'].apply(lambda x: 1 if x!='nan' else 0)

In [60]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

In [61]:
new_test_valid.head(1)

,Unnamed: 0,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,prediction_extractions,main extraction prediction,meddra classification prediction,soft_medra_classification,hard_medra_classfication,extraction binarized,meddra_term binarized,soft_ext_classification,hard_ext_classfication,true_ext
0,0.0,3.323175e+17,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,[],0,0,0,0,1,1,0,0,1


In [58]:
# Final Relaxed - MedDRA
print(classification_report(new_test_valid['meddra_term binarized'], new_test_valid['soft_medra_classification']))
print("F1 score: ", f1_score(new_test_valid['meddra_term binarized'], new_test_valid['soft_medra_classification']))
print("Precision score: ", precision_score(new_test_valid['meddra_term binarized'], new_test_valid['soft_medra_classification']))
print("Recall score: ", recall_score(new_test_valid['meddra_term binarized'], new_test_valid['soft_medra_classification']))

              precision    recall  f1-score   support

           0       0.06      0.12      0.08       199
           1       0.20      0.11      0.15       392

    accuracy                           0.12       591
   macro avg       0.13      0.12      0.12       591
weighted avg       0.16      0.12      0.13       591

F1 score:  0.14705882352941177
Precision score:  0.20454545454545456
Recall score:  0.11479591836734694


In [33]:
# Final Strict - MedDRA
print(classification_report(new_test_valid['meddra_term binarized'], new_test_valid['hard_medra_classfication']))
print("F1 score: ", f1_score(new_test_valid['meddra_term binarized'], new_test_valid['hard_medra_classfication']))
print("Precision score: ", precision_score(new_test_valid['meddra_term binarized'], new_test_valid['hard_medra_classfication']))
print("Recall score: ", recall_score(new_test_valid['meddra_term binarized'], new_test_valid['hard_medra_classfication']))

              precision    recall  f1-score   support

           0       0.06      0.12      0.08       199
           1       0.15      0.08      0.10       392

    accuracy                           0.09       591
   macro avg       0.10      0.10      0.09       591
weighted avg       0.12      0.09      0.09       591

F1 score:  0.10050251256281406
Precision score:  0.14634146341463414
Recall score:  0.07653061224489796


In [67]:
# Relaxed - NER
print(classification_report(new_test_valid['meddra_term binarized'], new_test_valid['soft_ext_classification']))
print("F1 score: ", f1_score(new_test_valid['meddra_term binarized'], new_test_valid['soft_ext_classification']))
print("Precision score: ", precision_score(new_test_valid['meddra_term binarized'], new_test_valid['soft_ext_classification']))
print("Recall score: ", recall_score(new_test_valid['meddra_term binarized'], new_test_valid['soft_ext_classification']))

              precision    recall  f1-score   support

           0       0.08      0.15      0.11       199
           1       0.29      0.18      0.22       392

    accuracy                           0.17       591
   macro avg       0.19      0.16      0.16       591
weighted avg       0.22      0.17      0.18       591

F1 score:  0.22151898734177217
Precision score:  0.2916666666666667
Recall score:  0.17857142857142858


In [68]:
# Strict - NER
print(classification_report(new_test_valid['meddra_term binarized'], new_test_valid['hard_ext_classfication']))
print("F1 score: ", f1_score(new_test_valid['meddra_term binarized'], new_test_valid['hard_ext_classfication']))
print("Precision score: ", precision_score(new_test_valid['meddra_term binarized'], new_test_valid['hard_ext_classfication']))
print("Recall score: ", recall_score(new_test_valid['meddra_term binarized'], new_test_valid['hard_ext_classfication']))

              precision    recall  f1-score   support

           0       0.08      0.15      0.10       199
           1       0.24      0.14      0.18       392

    accuracy                           0.14       591
   macro avg       0.16      0.14      0.14       591
weighted avg       0.19      0.14      0.15       591

F1 score:  0.1753246753246753
Precision score:  0.24107142857142858
Recall score:  0.1377551020408163


In [41]:
# Hard Extaction
# print(classification_report(new_test_valid['extraction binarized'], new_test_valid['hard_medra_classfication']))

In [42]:
# Soft Classification
# print(classification_report(new_test_valid['meddra_term binarized'], new_test_valid['soft_medra_classification']))

In [40]:
# Soft Classification
# print(classification_report(new_test_valid['meddra_term binarized'], new_test_valid['hard_medra_classfication']))

In [ ]:
new_test_valid.to_excel('final output.xlsx',index=False)

0    386
1    205
dtype: int64